In [1]:
#coding:utf-8

In [2]:
# 跟「繪圖」有關的套件，如果沒有打算繪圖就不用 import
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.utils import shuffle
drive.mount('/content/drive')
import math
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Input, LSTM, Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import gensim.downloader

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def show_train_history(train_history):

  #fig , ax = plt.subplots()
  #fig.subplots_adjust(hspace=0.4, wspace=0.4) #設定子圖的間隔
  #fig.set_figwidth(14)
  """
  ax1 = plt.subplot(1, 2, 1)
  plt.title("Accuracy")
  plt.plot(train_history.history['accuracy'],'-', label='accuracy')
  plt.plot(train_history.history['val_accuracy'],'-',  label='val_accuracy')
  leg = ax1.legend(loc='lower right') 
  plt.ylabel('accuracy')
  plt.xlabel('Epoch')
  """

  #ax2 = plt.subplot(1, 2, 2)
  plt.title("Loss")
  plt.plot(train_history.history['loss'],'-', label='loss')
  plt.plot(train_history.history['val_loss'],'-', label='val_loss')
  #leg = ax2.legend(loc='upper right') 
  plt.ylabel('loss')
  plt.xlabel('Epoch')
  plt.show()

In [4]:
data = pd.read_csv("/content/drive/MyDrive/DLPFLP/name,outline,totalFundedUSD.csv")
data = data.drop('country_code', axis=1)
data = data.drop('funding_rounds', axis=1)
data = data.drop('status', axis=1)
data = data.drop('entity_id', axis=1)
data = data.drop('normalized_name', axis=1)
data = data.dropna()
def usdLogged(x):
  if x == 0:
    return 0
  else:
    return math.log(x, 10)

def normalize(x, max):
  if (x == 0):
    return 0
  else:
    return x / max

data['logged_funding_total_usd'] = data['funding_total_usd'].apply(lambda x:usdLogged(x))
max_logged_funding_total_usd = data['logged_funding_total_usd'].max()
data['normalized_logged_funding_total_usd'] = data['logged_funding_total_usd'].apply(lambda x:normalize(x, max_logged_funding_total_usd))
data['inverted_normalized_logged_funding_total_usd'] = data['normalized_logged_funding_total_usd'].apply(lambda x:1-x)
print(data['normalized_logged_funding_total_usd'])

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


0         0.77895
1         0.00000
2         0.00000
3         0.00000
4         0.00000
           ...   
462629    0.00000
462634    0.00000
462637    0.00000
462642    0.00000
462643    0.00000
Name: normalized_logged_funding_total_usd, Length: 235235, dtype: float64


In [5]:
trainfundingTotalUsd = data['logged_funding_total_usd'].to_numpy()
trainOverviewString = data['overview'].to_numpy()
print(trainfundingTotalUsd)
print(trainfundingTotalUsd.shape)
print(trainOverviewString.shape)

[7.59933713 0.         0.         ... 0.         0.         0.        ]
(235235,)
(235235,)


In [6]:
def split_list(target_list:list, percent:float):
  return target_list[0:int(len(target_list) * percent)], target_list[int(len(target_list) * percent):int(len(target_list))]

In [7]:
print(trainfundingTotalUsd.shape)
print(trainOverviewString.shape)
trainOverviewString, trainfundingTotalUsd = shuffle(trainOverviewString, trainfundingTotalUsd)
trainfundingTotalUsd, testfundingTotalUsd = split_list(trainfundingTotalUsd, 0.8)
trainOverviewString, testOverviewString = split_list(trainOverviewString, 0.8)
print(trainfundingTotalUsd.shape)
print(trainfundingTotalUsd)
print(trainOverviewString.shape)
print(trainOverviewString)
print(testfundingTotalUsd.shape)
print(testfundingTotalUsd)
print(testOverviewString.shape)
print(testOverviewString)

(235235,)
(235235,)
(188188,)
[0. 0. 0. ... 0. 0. 0.]
(188188,)
['Chris Grant is Co-Founder of Synthace.'
 'Kabir Sehgal is an entrepreneur, startup consultant, humanitarian, public and motivational speaker, career counselor, Google Employee, business owner in the Bay Area. \n\nHe is 21 years old, worked at Google as a Product Quality Associate and runs multiple student help groups and websites: Slug Jobs, Spartan Jobs, Santa Cruz Successful Scholars, and fieldstudy (www.fieldstudy.net)\n\nKabir has worked at Plantronics, Market Motive, Brush with Bamboo, Google, FieldStudy and Slug Jobs. \n\nKabir Sehgal has graduated from the University of California, Santa Cruz with a B.A. in Business Management Economics a full two years earlier than my classmates with the goal of accelerating my real-world learning. It?s my time to take big risks, make big moves, and learn along the way. I want to learn from passionate and intelligent teams working on game-changing projects in fields related to my

In [8]:
print(trainfundingTotalUsd.min())

0.0


In [9]:
# 非必要，但用了會比較快
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(" Number of GPUs available: " , len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

 Number of GPUs available:  1


In [10]:
# pretrained word embedding
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

In [11]:
print(len(glove_vectors['hello']))

50


In [12]:
for index in range(len(trainOverviewString)):
  trainOverviewString[index] = tf.keras.preprocessing.text.text_to_word_sequence(trainOverviewString[index])

for index in range(len(testOverviewString)):
  testOverviewString[index] = tf.keras.preprocessing.text.text_to_word_sequence(testOverviewString[index])



In [13]:
print(trainOverviewString[5])

['a', 'work', 'collaboration', 'network', 'for', 'those', 'with', 'multiple', 'projects', 'tasks', 'and', 'colleagues', 'to', 'manage', 'and', 'communicate', 'with', 'simple', 'and', 'intuitive', 'to', 'use', 'head', 'of', 'the', 'curve', 'integrates', 'web', 'conferencing', 'a', 'project', 'task', 'manager', 'and', 'a', 'simultaneous', 'text', 'editor', 'into', 'a', 'powerful', 'and', 'versatile', 'private', 'business', 'network', 'uk', 'based', 'from', 'free', 'through', 'two', 'subscription', 'levels', 'of', 'access', 'to', "'enterprise'", 'which', 'is', 'a', 'bespoke', 'service', 'giving', 'tariff', 'choices', 'multiple', 'networks', 'multi', 'users', 'and', 'branding', 'enterprise', 'also', 'provides', 'a', 'dedicated', 'account', 'manager', 'ensuring', 'direct', 'contact', 'at', 'all', 'times', 'for', 'queries', 'and', 'problems', 'uk', 'based', 'so', 'time', 'difference', 'will', 'apply', 'elsewhere', 'for', 'more', 'information', 'please', 'go', 'to', 'home', 'login', 'page', '

In [14]:
# self made tokenizer
# tokenizer = Tokenizer(lower=True, num_words=40000)
# tokenizer.fit_on_texts(trainOverviewString)

In [15]:
# trainOverviewString = tokenizer.texts_to_sequences(trainOverviewString)
# print(trainOverviewString[20909])

In [16]:
# trainOverviewString = pad_sequences(trainOverviewString, padding='post', maxlen=100)
# print(trainOverviewString[10])
# print(trainOverviewString.shape)

In [17]:
# num_words = len(tokenizer.word_index) + 1
# print(num_words)

In [18]:
model = Sequential([
  # Embedding(input_dim=40000, output_dim=16, mask_zero=True),

  LSTM(256, input_length=100, input_dim=50),
  Dropout(0.2),
  Dense(512, activation='relu'),
  Dropout(0.2),
  Dense(256, activation='relu'),
  Dropout(0.2),
  Dense(128, activation='relu'),
  Dropout(0.2),
  Dense(64, activation='relu'),
  Dropout(0.2),
  Dense(1, activation='linear')
])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               314368    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 512)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               3

In [20]:
model.compile(loss='huber_loss', optimizer=Adam(learning_rate=0.00001, decay=0.0001))

In [21]:
print(trainOverviewString)
print(trainfundingTotalUsd)

[list(['chris', 'grant', 'is', 'co', 'founder', 'of', 'synthace'])
 list(['kabir', 'sehgal', 'is', 'an', 'entrepreneur', 'startup', 'consultant', 'humanitarian', 'public', 'and', 'motivational', 'speaker', 'career', 'counselor', 'google', 'employee', 'business', 'owner', 'in', 'the', 'bay', 'area', 'he', 'is', '21', 'years', 'old', 'worked', 'at', 'google', 'as', 'a', 'product', 'quality', 'associate', 'and', 'runs', 'multiple', 'student', 'help', 'groups', 'and', 'websites', 'slug', 'jobs', 'spartan', 'jobs', 'santa', 'cruz', 'successful', 'scholars', 'and', 'fieldstudy', 'www', 'fieldstudy', 'net', 'kabir', 'has', 'worked', 'at', 'plantronics', 'market', 'motive', 'brush', 'with', 'bamboo', 'google', 'fieldstudy', 'and', 'slug', 'jobs', 'kabir', 'sehgal', 'has', 'graduated', 'from', 'the', 'university', 'of', 'california', 'santa', 'cruz', 'with', 'a', 'b', 'a', 'in', 'business', 'management', 'economics', 'a', 'full', 'two', 'years', 'earlier', 'than', 'my', 'classmates', 'with', 't

In [22]:
print(trainOverviewString.shape)
print(trainfundingTotalUsd.shape)
train_history = model.fit(trainOverviewString, trainfundingTotalUsd, validation_split=0.2, batch_size=250, epochs=30, callbacks=[EarlyStopping(monitor='val_loss', patience=1)])

(188188,)
(188188,)


ValueError: ignored

In [ ]:
show_train_history(train_history)

In [ ]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

In [ ]:
testOverviewString = tokenizer.texts_to_sequences(testOverviewString.tolist())
print(testOverviewString)
testOverviewString = pad_sequences(testOverviewString, padding='post', maxlen=100)
print(testOverviewString)


In [ ]:
predict = model.predict(testOverviewString)
print(predict)

In [ ]:
f = plt.figure()
f.set_figwidth(10)
f.set_figheight(10)
plt.plot([0, 10], [0, 10])
plt.scatter(x=predict, y=testfundingTotalUsd,s=0.5)

plt.show()